In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/mounts/work/faeze/.cache/hf/'
os.environ['HF_HOME'] = '/mounts/work/faeze/.cache/hf/'
os.environ['HF_DATASETS_CACHE'] = '/mounts/work/faeze/.cache/hf/'
os.environ['TORCH_HUB'] = '/mounts/work/faeze/.cache/torch/'
os.environ['TORCH_HOME'] = '/mounts/work/faeze/.cache/torch/'
os.environ["WANDB_DIR"] = '/mounts/work/faeze/.cache/wandb/'

In [2]:
from data_provider import DataProvider
from finetuner import FineTuner, FineTunerConfig

/mounts/Users/cisintern/faeze/.local/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
data_provider_ = DataProvider()

In [4]:
names = ['bas19_es', 'dyn21_en', 'for19_pt', 'fou18_en',
                       'has19_hi', 'has20_hi', 'has21_hi', 'ken20_en',
                       'ous19_ar', 'ous19_fr', 'san20_it',]

datas = [f'baseline_data/{i}' for i in names]
languages = [name.split('_')[1] for name in names]
languages

['es', 'en', 'pt', 'en', 'hi', 'hi', 'hi', 'en', 'ar', 'fr', 'it']

In [5]:
datasets = data_provider_.load_datasets(datas, languages)

/mounts/work/faeze/new_miniconda3/envs/2024_9/lib/python3.10/site-packages/datasets/load.py:929: FutureWarning: The repository for baseline_data contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at baseline_data/baseline_data.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [6]:
datasets[5]

{'name': 'baseline_data/dyn21_en',
 'split': 'test',
 'data': Dataset({
     features: ['id', 'text', 'label'],
     num_rows: 2000
 }),
 'language': 'en'}

In [7]:
train_data = datasets[0]['data']
eval_data = datasets[1]['data']
test_data = datasets[2]['data']

In [10]:


config = FineTunerConfig(
        model_name="google/mt5-base",
        fine_tune_method="lora",
        num_labels=2,  # Binary classification (Hate Speech detection)
        learning_rate=5e-5,
        epochs=3,
        batch_size=16,
    # peft_config={"lora_rank": 4}
    )

In [11]:
fine_tuner = FineTuner(config)

Some weights of MT5ForSequenceClassification were not initialized from the model checkpoint at google/mt5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
train_dataset = fine_tuner.prepare_data(train_data)
eval_dataset = fine_tuner.prepare_data(eval_data)
test_dataset = fine_tuner.prepare_data(test_data)

In [13]:
fine_tuner.train(train_dataset, eval_dataset)

/mounts/Users/cisintern/faeze/.local/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: parvaz. Use `wandb login --relogin` to force relogin


/mounts/Users/cisintern/faeze/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,0.735400,No log


KeyError: "The `metric_for_best_model` training argument is set to 'eval_loss', which is not found in the evaluation metrics. The available evaluation metrics are: []. Consider changing the `metric_for_best_model` via the TrainingArguments."

In [17]:
prediction = fine_tuner.predict(test_dataset)
results = fine_tuner.compute_metrics(prediction)
print(results)

/mounts/Users/cisintern/faeze/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'accuracy': 0.758, 'f1-macro': 0.715371469635899, 'precision': 0.7543940303940304, 'recall': 0.758, 'f1-weighted': 0.7559071264162328}


In [ ]:
{'accuracy': 0.782, 'f1-macro': 0.7790265816831281, 'precision': 0.7872806227483272, 'recall': 0.782, 'f1-weighted': 0.7831278483270894}

In [ ]:
{'accuracy': 0.758, 'f1-macro': 0.715371469635899, 'precision': 0.7543940303940304, 'recall': 0.758, 'f1-weighted': 0.7559071264162328}